## G -Part

### Imports
*Initialize the environment and the Linear Regression class*

In [1]:
from lin_reg import LinearRegression
import numpy as np

datafile = "housing.csv"
model = LinearRegression()

np.set_printoptions(precision=2, suppress=True, linewidth=100)

data = np.genfromtxt(datafile, delimiter=",", names=True, dtype=None, 
                     encoding="utf-8",  missing_values="", filling_values=np.nan)

### Cleaning & Organizing
*Remove incomplete rows*

In [2]:
target_y = "median_house_value"
num_features = [
    "longitude", "latitude", "housing_median_age", 
    "total_rooms", "total_bedrooms", "population", 
    "households", "median_income"
]
cat_features = ["ocean_proximity"]
all_features = [target_y] + num_features + cat_features

mask = np.ones(len(data), dtype=bool)

for col in all_features:
    if data[col].dtype.kind in "fi":
        mask &= ~np.isnan(data[col])
    else: 
        mask &= (data[col] != "") & (data[col] != "nan")

clean_data = data[mask]

### Extra - Cleaning stats
*Summary of the data lost during the filtering process*

In [3]:
missing_results = []
for col in all_features:
    if data[col].dtype.kind in "fi":
        missing_count = np.sum(np.isnan(data[col]))
    else:
        missing_count = np.sum((data[col] == "") | (data[col] == "nan"))
    missing_results.append(f"{col}: {missing_count} missing values")
(missing_results, {
    "Total rows before": len(data),
    "Total rows remaining": len(clean_data),
    "Total rows lost": len(data) - len(clean_data)
})

(['median_house_value: 0 missing values',
  'longitude: 0 missing values',
  'latitude: 0 missing values',
  'housing_median_age: 0 missing values',
  'total_rooms: 0 missing values',
  'total_bedrooms: 207 missing values',
  'population: 0 missing values',
  'households: 0 missing values',
  'median_income: 0 missing values',
  'ocean_proximity: 0 missing values'],
 {'Total rows before': 20640,
  'Total rows remaining': 20433,
  'Total rows lost': 207})

### Categorical Features
*Transform text-based location data into binary numerical features*

In [4]:
unique_cats = np.unique(clean_data["ocean_proximity"])

cats_to_encode = unique_cats[:-1]
encoded_list = []

for cat in cats_to_encode:
    binary_col = np.zeros(len(clean_data))
    
    for i in range(len(clean_data)):
        if clean_data["ocean_proximity"][i] == cat:
            binary_col[i] = 1
    
    encoded_list.append(binary_col)

ocean_encoded = np.column_stack(encoded_list)
ocean_encoded.shape

(20433, 4)

### Preparing the data for the model
*Constructing the feature matrix X and the target vector y*

In [5]:
y = clean_data[target_y]
num_cols = []
for feature in num_features:
    num_cols.append(clean_data[feature])
X_num = np.column_stack(num_cols)

X_all = np.hstack([X_num, ocean_encoded])
intercept = np.ones(len(y))
X = np.column_stack([intercept, X_all])

### Train and use the model
*Executing the OLS fit and calculating standard error metrics*

In [6]:
model.fit(X,y)

variance = model.variance_calc(X,y)
std_dev = model.st_dev_calc(X,y)
rmse = model.rmse_calc(X,y)

[
    f"n (sample size): {model.n}",
    f"d (features): {model.d}",
    f"Variance (σ²): {variance:.2f}",
    f"Standard Deviation: {std_dev:.2f}",
    f"RMSE: {rmse:.2f}"
]


['n (sample size): 20433',
 'd (features): 12',
 'Variance (σ²): 4713776929.50',
 'Standard Deviation: 68656.95',
 'RMSE: 68635.11']

## VG Part

### $R^2$
*Evaluating how much of the variation in house prices the model explains.*

In [7]:
r_squared = model.r_squared_calc(X,y)
f"R-squared: {r_squared:.4f}"

'R-squared: 0.6465'

### F-Statistic & p-value
*Testing if the group of predictors together helps predict price better than using no predictors.*

In [8]:
f_stat, p_value = model.f_test(X,y)
{
    "F-Statistic": round(float(f_stat), 2),
    "p-value": f"{float(p_value):.10f}"
}

{'F-Statistic': 3111.61, 'p-value': '0.0000000000'}

### Pearson Correlation
*Analyzing linear relationships between feature pairs*

In [9]:
pearson = model.pearson_corr(X)
feature_names = ['intercept', 'longitude', 'latitude', 'housing_median_age', 
                 'total_rooms', 'total_bedrooms', 'population', 'households',
                 'median_income', 'ocean_<1H', 'ocean_INLAND', 'ocean_ISLAND', 
                 'ocean_NEAR_BAY']
short_names = ["LON", "LAT", "AGE", "RMS", "BED", "POP", "HHD", "INC", "<1H", "INL", "ISL", "BAY"]
rows = [f"{feature_names[i+1]:20s}: {pearson[i]}" for i in range(len(pearson))]

footer = " " * 24 + "   ".join(short_names)
rows.append(footer)
rows

['longitude           : [ 1.   -0.92 -0.11  0.05  0.07  0.1   0.06 -0.02  0.32 -0.06  0.01 -0.47]',
 'latitude            : [-0.92  1.    0.01 -0.04 -0.07 -0.11 -0.07 -0.08 -0.45  0.35 -0.02  0.36]',
 'housing_median_age  : [-0.11  0.01  1.   -0.36 -0.32 -0.3  -0.3  -0.12  0.05 -0.24  0.02  0.26]',
 'total_rooms         : [ 0.05 -0.04 -0.36  1.    0.93  0.86  0.92  0.2  -0.    0.03 -0.01 -0.02]',
 'total_bedrooms      : [ 0.07 -0.07 -0.32  0.93  1.    0.88  0.98 -0.01  0.02 -0.01 -0.   -0.02]',
 'population          : [ 0.1  -0.11 -0.3   0.86  0.88  1.    0.91  0.01  0.07 -0.02 -0.01 -0.06]',
 'households          : [ 0.06 -0.07 -0.3   0.92  0.98  0.91  1.    0.01  0.04 -0.04 -0.01 -0.01]',
 'median_income       : [-0.02 -0.08 -0.12  0.2  -0.01  0.01  0.01  1.    0.17 -0.24 -0.01  0.06]',
 'ocean_<1H           : [ 0.32 -0.45  0.05 -0.    0.02  0.07  0.04  0.17  1.   -0.61 -0.01 -0.31]',
 'ocean_INLAND        : [-0.06  0.35 -0.24  0.03 -0.01 -0.02 -0.04 -0.24 -0.61  1.   -0.01 -0.24]',


### Discussion : Collinearity and parameters selection
The Pearson correlation table shows that several predictors share very similar information. The strongest group is formed by the housing related variables: **total_bedrooms** correlates  with **households** at 0.98,**total_rooms** with **total_bedrooms** at 0.93, and with **households** at 0.92. These values are extremely high, which means these variables move almost in lockstep. **Population** is also closely tied to this group, with correlations above 0.85 with all 3 of them. Together, these four variables describe nearly the same underlying pattern in the dataset.

There is also a very strong relationship between the geographic variables longitude and latitude (−0.92). This indicates that the locations in the dataset follow a narrow diagonal pattern, so the two coordinates do not vary independently.

**Statistical Impact:**
When predictors are highly correlated, the variance-covariance matrix c = (X^T X)^(-1)σ² 
has inflated diagonal elements (cᵢᵢ). Since SE(B̂ᵢ) = σ̂ √cᵢᵢ, this directly increases 
standard errors. The model struggles to solve (X^T X)b = X^T y reliably when X^T X is 
near-singular due to collinearity.

Because these predictors overlap so much, the model cannot fully separate their individual effects. When several variables carry almost the same information, the coefficient of one variable can easily change sign depending on which of the others are included. This explains why total_rooms appears with the “wrong” sign: the model is trying to distribute a shared signal across several nearly identical predictors, and the sign becomes unstable as a result. This behaviour is typical in the presence of strong collinearity and does not mean that the variable has a genuinely negative relationship with the outcome.

These correlations do not create a statistical hierarchy in the regression sense. A statistical hierarchy would require keeping certain variables because they appear inside interaction terms or polynomial terms. Here, the predictors are correlated, but none of them are mathematically built from the others. The correlations therefore do not force any variable to remain in the model for hierarchical reasons.
Statistical hierarchy applies when:
- Interaction: Y = β₀ + β₁X₁ + β₂X₂ + β₃X₁X₂ (must keep X₁, X₂ if X₁X₂ is significant)
- Polynomial: Y = β₀ + β₁X + β₂X² (must keep X if X² is significant)

Our model has only raw variables, so no hierarchical constraint applies.
The housing variables do reflect different levels of the same structure which explains why they correlate so strongly. This structure justifies including them initially if the goal is to describe the housing environment in detail. However, their strong overlap limits how separately their coefficients can be interpreted. The unusual sign of total_rooms is a direct consequence of this overlap rather than a meaningful effect.

In short, the Pearson correlations show strong collinearity but no statistical hierarchy that forces us to keep variables in the model. The housing variables form a tightly connected cluster, which justifies their initial inclusion, but their overlap explains both the instability of coefficients and the unexpected sign of total_rooms. If the goal is clearer and more stable effects, simplifying this group is appropriate; if the goal is descriptive detail, the variables can be kept, as long as their dependence is acknowledged.

### T-tests
*Testing whether a single predictor has a real effect on price (different from zero).*

In [10]:
t_stats, p_values = model.t_tests(X,y)
results =  []


for i in range(len(t_stats)):
    sig = "Significant" if p_values[i] <0.05 else "NOT significant"
    results.append(f"{feature_names[i]:20s}: {t_stats[i]:8.4f}, {p_values[i]:.6f}")

results


['intercept           : -25.6395, 0.000000',
 'longitude           : -26.2963, 0.000000',
 'latitude            : -25.3629, 0.000000',
 'housing_median_age  :  24.4389, 0.000000',
 'total_rooms         :  -7.8250, 0.000000',
 'total_bedrooms      :  14.6402, 0.000000',
 'population          : -35.2824, 0.000000',
 'households          :   6.6589, 0.000000',
 'median_income       : 116.1510, 0.000000',
 'ocean_<1H           :  -2.7258, 0.006421',
 'ocean_INLAND        : -19.3633, 0.000000',
 'ocean_ISLAND        :   4.8327, 0.000001',
 'ocean_NEAR_BAY      :  -3.7835, 0.000155']

All predictors in the fitted model are statistically detectable at the 5% level (every predictor’s p‑value < 0.05). The t‑tests identify a set of variables that the model finds informative for predicting house price in this sample.
With a sample size of 20,433, even small effects are easily detected as statistically significant, so the p-values alone don't indicate which predictors are most important—the t-statistics provide better ranking of effect strength.

The strongest evidence is for **median_income** (t = 116.15, p < 0.001), which shows a very large positive association with house price in the sample;

**population** (t = −35.28, p < 0.001) shows a strong negative association; 

**longitude** (t = −26.30, p < 0.001) and **latitude** (t = −25.36, p < 0.001) both show large negative associations in this fit, and 

**housing_median_age** (t = 24.44, p < 0.001) also has a strong positive association. 

**total_rooms** has a negative t (t = −7.83) in the original model despite an intuitive expectation of a positive effect; 
This can be because of thr high overlapping data with *total_bedrooms*, *households* and *population* as we've seen in the Paerson correlation table. This can cause instability in individual coefficients resulting in imprecise values or signs. 

Among the categorical proximity indicators, **ocean_INLAND** is strongly negative (t = −19.3633, p < 0.001), **ocean_ISLAND** is positive (t = 4.8327, p < 0.001), **ocean_NEAR_BAY** is negative (t = −3.7835, p ≈ 0.00016), and **ocean_<1H** is detectable with weaker evidence (t = −2.7258, p ≈ 0.0064). 

*Confidence intervals (in the next cell) agree with the t‑tests: none include zero, which supports the significance results above.*

### Confidence Intervals
*Gives a plausible range for the true effect of a predictor; if the range does not include zero, the effect is detectable.*

In [11]:
lower, upper = model.confidence_interval(X,y,alpha=0.05)
ci_table = {}

for i in range(len(feature_names)):
    ci_table[feature_names[i]] = {
        "Lower": round(lower[i], 2),
        "Upper": round(upper[i], 2)
    }

ci_table


{'intercept': {'Lower': -2438881.36, 'Upper': -2092470.6},
 'longitude': {'Lower': -28811.59, 'Upper': -24814.39},
 'latitude': {'Lower': -27451.48, 'Upper': -23512.89},
 'housing_median_age': {'Lower': 986.5, 'Upper': 1158.54},
 'total_rooms': {'Lower': -7.74, 'Upper': -4.64},
 'total_bedrooms': {'Lower': 87.09, 'Upper': 114.02},
 'population': {'Lower': -40.08, 'Upper': -35.86},
 'households': {'Lower': 35.01, 'Upper': 64.22},
 'median_income': {'Lower': 38597.06, 'Upper': 39922.09},
 'ocean_<1H': {'Lower': -7354.53, 'Upper': -1201.74},
 'ocean_INLAND': {'Lower': -47972.11, 'Upper': -39152.75},
 'ocean_ISLAND': {'Lower': 88344.04, 'Upper': 208903.57},
 'ocean_NEAR_BAY': {'Lower': -12496.91, 'Upper': -3967.47}}

-------------------------------------------------------------------------------------------------------- 

### Discussion on confidence Levels:

For this analysis, I use a 95% confidence level (α=0.05). This indicates that I am accepting a 5% risk of a Type I error, specifically, the risk of rejecting the null hypothesis (concluding a coefficient is significant) when it is actually zero in the population.

Since the model has R² = 0.65, it explains a reasonable portion of the variation 
in house prices but leaves 35% unexplained. This moderate fit supports using the 
standard 95% level—it's strict enough to avoid false conclusions but not so strict 
that the intervals become too wide to be useful. 

A 90% level would make the intervals narrower but less reliable, while a 99% level would make them so wide that it becomes difficult to draw useful conclusions. A 95% level provides a reasonable balance between these two extremes.

Looking at the resulting confidence intervals, a few predictors stand out as clearly influential. Median_income has a strongly positive interval(38597,  39922), which shows that higher income is associated with higher predicted prices. Several location related variables also show stable effects: both longitude and latitude have negative intervals, indicating a relationship between location and price in the dataset. Among the categorical variables, ocean_ISLAND has a large positive interval (88344,  208903), while ocean_INLAND shows a clear negative effect(-47972.11, -39152.75).

Other predictors have smaller but still consistent intervals. For example, total_rooms and population both have narrow negative intervals, suggesting small downward effects. Total_bedrooms and households show modest positive intervals. These effects are weaker than the main predictors but still statistically meaningful since their intervals do not cross zero.

The width of each interval reflects how precisely the coefficient is estimated: narrower intervals indicate more certainty, while wider ones show more uncertainty. Since none of the intervals include zero, each predictor shows a statistically reliable direction of effect at the chosen confidence level.


### Extra - Analyzing high correlations:
*Highest multicollinearity*

In [12]:
corr_matrix = model.pearson_corr(X)
actual_features = feature_names[1:]
high_corr_list = []

for i in range(len(actual_features)):
    for j in range(i + 1, len(actual_features)):
        correlation = corr_matrix[i, j]
        if abs(correlation) > 0.85:
            high_corr_list.append((actual_features[i], actual_features[j], correlation))

high_corr_list.sort(key=lambda x: abs(x[2]), reverse=True)

[f"{feat_a:14s} & {feat_b:18s}: {val:.2f}" for feat_a, feat_b, val in high_corr_list]

['total_bedrooms & households        : 0.98',
 'total_rooms    & total_bedrooms    : 0.93',
 'longitude      & latitude          : -0.92',
 'total_rooms    & households        : 0.92',
 'population     & households        : 0.91',
 'total_bedrooms & population        : 0.88',
 'total_rooms    & population        : 0.86']

*Highest significance*

In [13]:
t_stats, _ = model.t_tests(X, y)

impact_list = []
for i in range(1, len(feature_names)):
    impact_list.append((feature_names[i], abs(t_stats[i])))

impact_list.sort(key=lambda x: x[1], reverse=True)
[f"{name:18s} | {val:10.2f}" for name, val in impact_list[:5]]

['median_income      |     116.15',
 'population         |      35.28',
 'longitude          |      26.30',
 'latitude           |      25.36',
 'housing_median_age |      24.44']

### Extra - Refined Model demonstration

In [14]:
refined_indices = [0,1,2,3,4,6,8,9,10,11,12]
X_refined = X[:, refined_indices]

model_refined = LinearRegression()
model_refined.fit(X_refined, y)

[
 f"Unrefined RMSE: {rmse:.2f}" , 
 f"Unrefined Variance (σ²): {variance:.2f}",  
 f"Refined RMSE: {model_refined.rmse_calc(X_refined, y):.2f}", 
 f"Refined Variance (σ²): {model_refined.variance_calc(X_refined, y):.2f}"
 ]

['Unrefined RMSE: 68635.11',
 'Unrefined Variance (σ²): 4713776929.50',
 'Refined RMSE: 70690.98',
 'Refined Variance (σ²): 4999906984.29']

In [15]:
t_stats_ref, p_values_ref = model_refined.t_tests(X_refined, y)
ref_feature_names = [
    'intercept', 'longitude', 'latitude', 'housing_median_age', 
    'total_rooms', 'population', 'median_income', 
    'ocean_<1H', 'ocean_INLAND', 'ocean_ISLAND', 'ocean_NEAR_BAY'
]

original_results = ["--- ORIGINAL MODEL ---"]
for i in range(len(t_stats)):
        original_results.append(f"{feature_names[i]:20s}: {t_stats[i]:8.4f}, {p_values[i]:.6f}")
refined_results = ["--- REFINED MODEL ---"]
for i in range(len(t_stats_ref)):
        refined_results.append(f"{ref_feature_names[i]:20s}: {t_stats_ref[i]:8.4f}, {p_values_ref[i]:.6f}")

comparison = original_results + [""] + refined_results
comparison

['--- ORIGINAL MODEL ---',
 'intercept           : -25.6395, 0.000000',
 'longitude           : -26.2963, 0.000000',
 'latitude            : -25.3629, 0.000000',
 'housing_median_age  :  24.4389, 0.000000',
 'total_rooms         :  -7.8250, 0.000000',
 'total_bedrooms      :  14.6402, 0.000000',
 'population          : -35.2824, 0.000000',
 'households          :   6.6589, 0.000000',
 'median_income       : 116.1510, 0.000000',
 'ocean_<1H           :  -2.7258, 0.006421',
 'ocean_INLAND        : -19.3633, 0.000000',
 'ocean_ISLAND        :   4.8327, 0.000001',
 'ocean_NEAR_BAY      :  -3.7835, 0.000155',
 '',
 '--- REFINED MODEL ---',
 'intercept           : -25.9566, 0.000000',
 'longitude           : -27.0413, 0.000000',
 'latitude            : -26.2918, 0.000000',
 'housing_median_age  :  22.2453, 0.000000',
 'total_rooms         :  30.6387, 0.000000',
 'population          : -27.5044, 0.000000',
 'median_income       : 111.6947, 0.000000',
 'ocean_<1H           :  -2.4349, 0.014905

The refined model confirms that the unusual behaviour in the original regression was caused by multicollinearity. When several predictors describe almost the same underlying pattern, the model cannot assign their effects independently, and the coefficients become unstable. This is why total_rooms appeared with a negative sign despite representing a feature that should logically increase housing value. After removing the most overlapping variables (total_bedrooms and households), the remaining predictors no longer compete for the same signal, and total_rooms returns to a positive and more interpretable effect.

The refined model has a slightly higher RMSE and variance, which means its predictions are less precise. This happens because removing redundant predictors reduces the model’s ability to “fit” the sample perfectly. However, the trade‑off is beneficial for interpretation. The refined model produces coefficients that are more stable, easier to interpret, and less influenced by noise introduced by collinearity. For understanding how each predictor relates to housing prices, the refined model is therefore the more trustworthy one, while the full model remains better only for raw predictive accuracy.
